In [2]:
DO_TOKENIZATION=False
TRAIN_MODEL=True

In [3]:
from datasets import load_dataset
from itertools import chain
import tokenizers
import tqdm


import os
import os.path
import random
import torch

import torch.nn as nn
from torch.nn import functional as F

In [4]:
if DO_TOKENIZATION:
    tinystories_ds = load_dataset("roneneldan/TinyStories")
    datafiles = [f'stories_{i:02d}.json' for i in range(1, 50)]
    tinystories_sp_ds = load_dataset("robrenaud/multilingual_tinystories", data_files=datafiles)

In [5]:
if DO_TOKENIZATION:
    with open('train.txt', 'w') as full_stories_output :
        for story in tqdm.tqdm(tinystories_sp_ds['train']['story']):
            full_stories_output.write(story)
        
        for story in tqdm.tqdm(tinystories_ds['train']['text']):
            full_stories_output.write(story)

In [6]:
if DO_TOKENIZATION:
    tokenizer = tokenizers.ByteLevelBPETokenizer()

    tokenizer.train(files=['train.txt'], vocab_size=2**13, min_frequency=2)

    tokenizer.save_model('.', 'tiny-stories-bpe')
else:
    # still need to load the tokenizer
    tokenizer = tokenizers.ByteLevelBPETokenizer(
        "./tiny-stories-bpe-vocab.json", 
        "./tiny-stories-bpe-merges.txt")

In [7]:
if DO_TOKENIZATION:
    stories = chain(tinystories_ds['train']['text'], tinystories_sp_ds['train']['story'])
    
    if not os.path.isdir('tokenized'):
        os.mkdir('tokenized')
    output_buf = []
    num_outputs = 0
    for story in tqdm.tqdm(stories):
        encoded = torch.tensor(tokenizer.encode(story).ids, dtype=torch.short)
        output_buf.append(encoded)
        if len(output_buf) > 500_000:
            torch.save(output_buf, f'tokenized/tokenized-{num_outputs}.pt')
            num_outputs += 1
            output_buf = []
    if output_buf:
        torch.save(output_buf, f'tokenized/tokenized-{num_outputs}.pt')
        num_outputs += 1
        output_buf = []

## Train

In [8]:
#----- imports --------
import tqdm
import torch
# import wandb
import os
import tokenizers


device= 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device)
assert device == 'cuda', "This notebook is not optimized for CPU"

config = {
    "learning_rate": 1e-3,
    "eval_interval": 300,
    "max_iters": 60000, 
    "H": 32, # per head dimension size
    "B": 64, # batch size
    "T": 256, # Sequence length
    "C": 256, # model size
    "feedforward_factor": 3,
    "n_heads": 8,
    "dropout": 0.0,
    "l2_penalty": 0.0,
    "n_layers": 12,
    "tokenizer_vocab_size": 2**13,
    # "git_hash": os.popen("git rev-parse HEAD").read().strip()
}

# initial
for k,v in config.items():
    locals ()[k] = v

In [9]:
def load_sharded_story(shard_no):
    return torch.load(f'tokenized/tokenized-{shard_no}.pt')

In [10]:
# load the tokenized stories in parallel using threads
# this is faster than loading them sequentially
num_shards = len(os.listdir('tokenized'))
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor() as pool:
    stories = list(tqdm.tqdm(pool.map(load_sharded_story, range(num_shards)), total=num_shards))

100%|██████████| 15/15 [07:01<00:00, 28.09s/it]  


In [11]:
all_stories = []
for story in stories:
    all_stories.extend(story)
random.seed(1337)
random.shuffle(all_stories)

In [12]:
print("length of dataset in stories: ", len(all_stories))
print("length of stories in tokens", sum(len(story) for story in all_stories))

length of dataset in stories:  7019719
length of stories in tokens 1348019169


In [13]:
num_stories_to_check = 1_000_000
num_long = sum(len(story) > T for story in all_stories[:num_stories_to_check])
print(
    f"# stories longer than {T} : {num_long} out of {num_stories_to_check}, {num_long/num_stories_to_check:.2%}")

# stories longer than 256 : 102027 out of 1000000, 10.20%


In [14]:
def encode(text):
    return tokenizer.encode(text).ids
def decode(encoded_text):
    return tokenizer.decode(encoded_text)

from tqdm import tqdm

def batch_encode(text, batch_size):
    tokens = []
    for i in tqdm(range(0, len(text), batch_size)):
        tokens.extend(encode(text[i:i+batch_size]))
    return tokens


hello_encoded = encode("Hola")
print(hello_encoded)
print(decode(hello_encoded))
vocab_size = tokenizer.get_vocab_size()
print("vocab size: ", vocab_size)
print('first story decoded: ', decode(all_stories[0].tolist()))
PADDING_TOKEN_IDX= encode(" ")[0]

[718]
Hola
vocab size:  8192
first story decoded:  Un perro llamado Spot corría. Spot corría rápido. Él jugaba en el parque. Luego, ¡un trueno! Spot se asustó. 

Spot corrió a casa. Mamá lo vio. "¡Spot!", dijo Mamá. "¡No tengas miedo!" Spot se acurrucó con Mamá. Ella lo abrazó.

Después, ¡otro trueno! ¡Pero no era un trueno! Era Papá. Papá se reía. Él llevaba un gorro de trueno. "¡Sorpresa!", dijo Papá. 

Spot se rió. ¡No era un trueno malo! Era solo Papá. Él corrió a abrazar a Papá. Spot estaba feliz. 

Mamá sonrió. Spot era un perro feliz. Él amaba jugar con Papá. ¡Y amaba los truenos, ahora que sabía que eran solo Papá! 



In [15]:
n = int(0.9*len(all_stories))

train_data = all_stories[:n]
val_data = all_stories[n:]

In [16]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data), (B,)) 
    # ix = [i for i in range(B)]
    
    x = torch.full((B, T), PADDING_TOKEN_IDX, dtype=torch.long)
    y = torch.full((B, T), PADDING_TOKEN_IDX, dtype=torch.long)

    for sequence_index, random_story_index in enumerate(ix):
        story = data[random_story_index].long()[:T - 1]
        x[sequence_index][1: story.shape[0]+1] = story
        y[sequence_index][: story.shape[0]] = story

    return x, y

xb, yb = get_batch('train')

print(xb[0])
print(yb[0])

tensor([ 220,  582,  723,   11,  297,  342,  630,  560,  431,  297,  919,  782,
          13,  313,  569,  297,  782, 3210,  345, 2494,    0,  560,  440, 3610,
        1157,   13,  313, 1622, 1065,  443,    0,  411,  936,   11,  350,  956,
         782,  335,  312,  198, 1016,  375,  378,  322,   11,  292, 1655,  719,
         297, 1164,   11,  560,   11,  740,  366, 1157,  483,  560,  346,  471,
          13,  411,  366,  440, 1189,  340,  782,   13,  312,  198, 1865,   11,
         340,  375,  378,  749,  256,  560,  297,  919,  934,   13,  313,  569,
         297,  744,  725,    0,  484,  560,  378, 1273,  269,  315,  744,   13,
         350,  954,   11,  375,  335,  936,   13,  312,  198, 1126, 2374, 1157,
        1561,  315,  782,   13, 3105,  351,  340,  744,  310,  326,  792,   13,
         405,  375,  345,  560,  666,   13,  458,  833,  377,  560,  346,  443,
          13,  313,  278,  782,  366,  397, 1065, 1892, 2081,  345, 1157,    0,
         198,  220,  220,  220,  220,  2

In [17]:
class Head(nn.Module):
    '''One Head of self-attention'''
    def __init__(self, H):
        super().__init__()
        self.query = nn.Linear(C, H, bias=False)
        self.key = nn.Linear(C, H, bias=False)
        self.value = nn.Linear(C, H, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(T, T)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        query_vectors = self.query(x)
        key_vectors = self.key(x)


        # Attention masking(so we can't look into the past):
        tril = self.tril
        wei = torch.zeros(T, T) 
        wei = wei.masked_fill(tril == 0, float('-inf')) # set the upper triangular to -inf

        # multiply the two to get the attention weights
        attention_pattern = query_vectors @ key_vectors.transpose(-2, -1) # T, T
        attention_pattern = attention_pattern / (H ** 0.5) # scale the attention pattern for numerical stability
        attention_weights = F.softmax(attention_pattern + wei, dim=-1) # T, T (the row dimension is the query)
        attention_weights = self.dropout(attention_weights)

        value_vectors = self.value(x) # the direction we should go in the embedding space for each token (ie more blue) T, H

        # apply the attention weights to the value vectors
        context = attention_weights @ value_vectors # T, H

        # project back into original space from value space
        return context

x = torch.randn(B,T,C)
head = Head(H)

In [18]:
class MultiHeadAttention(nn.Module):
    '''Multiple heads of self-attention'''
    def __init__(self, H, C, n_heads): # H is head embedding space size, n_heads is number of heads
        super().__init__()
        self.heads = nn.ModuleList([Head(H) for _ in range(n_heads)])
        self.combine_heads = nn.Linear(H*n_heads, C)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        x = torch.cat([head(x) for head in self.heads], dim=-1)
        x = self.combine_heads(x)  # T, C
        return self.dropout(x)

In [19]:
head = MultiHeadAttention(H, C, n_heads)
head.heads[0].forward(x).shape

torch.Size([64, 256, 32])

In [20]:
class FeedForward(nn.Module):
    '''Feed-forward neural network'''
    def __init__(self, C):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(C, C * feedforward_factor),
            nn.ReLU(),
            nn.Linear(C * feedforward_factor, C),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [21]:
class LayerNorm(nn.Module):
    '''Layer normalization'''
    def __init__(self, C, use_affine=True):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(C)) if use_affine else None
        self.beta = nn.Parameter(torch.zeros(C)) if use_affine else None
        
    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        if self.gamma is not None and self.beta is not None:
            return self.gamma * (x - mean) / (std + 1e-6) + self.beta
        else:
            return (x - mean) / (std + 1e-6)

In [22]:
class Block(nn.Module):
    '''Transformer block'''
    def __init__(self, H, C, n_heads):
        super().__init__()
        self.attention = MultiHeadAttention(H, C, n_heads)
        self.ff = FeedForward(C)
        self.norm1 = LayerNorm(C, use_affine=True)
        self.norm2 = LayerNorm(C, use_affine=True)

    def forward(self, x):
        x = x + self.attention(self.norm1(x))
        x = x + self.ff(self.norm2(x))
        return x

In [23]:
class GPT(nn.Module):
    
    def __init__(self, n_layers):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, C)
        self.position_embedding_table = nn.Embedding(T, C)
        self.lm_head = nn.Linear(C, vocab_size)
        self.layers = nn.ModuleList([Block(H, C, n_heads) for _ in range(n_layers)])
        self.block = nn.ModuleList([Block(H, C, n_heads)])
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        token_emb = self.token_embedding_table(idx) # batch_dim, sequence_dim, embedding_dim
        pos_emb = self.position_embedding_table(torch.arange(T))
        x = token_emb + pos_emb # token identities and positions contained

        for layer in self.layers:
            x = layer(x)

        logits = self.lm_head(x) # batch_dim, sequence_dim, vocab_size

        batch_dim, sequence_dim, embedding_dim = logits.size()

        if targets is None:
            return logits, None
        else:
            logits_loss_view = logits.view(-1, vocab_size) 
            targets_loss_view = targets.view(-1)
            loss = F.cross_entropy(logits_loss_view, targets_loss_view)
            return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx[:,-T:])
            # get the predictions of the last token
            last_token_logits = logits[:, -1, :] # all batches, last token, all probabilities
            # softmax to get probabilities
            probabilities = F.softmax(last_token_logits, dim=-1)
            # sample from the probabilities
            next_token = torch.multinomial(probabilities, num_samples=1)
            # add the new token to the idx tensor
            idx = torch.cat((idx, next_token), dim=1)
        return idx
    def prompt_model(self, prompt, max_new_tokens, temperature=0.5):
        autoregressive_seq = encode(prompt)
        for _ in range(max_new_tokens):
            prediction_index = len(autoregressive_seq)-1

            model_input = torch.tensor(autoregressive_seq)
            
            while model_input.shape[0] < T:
                pad_token = torch.tensor(encode("\n"))
                model_input = torch.cat((model_input, pad_token), dim=0)

            model_input
            model_input = model_input.unsqueeze(0)

            logits, loss = model(model_input)
            prediction_token = logits[:, prediction_index, :] / temperature
            probabilities = F.softmax(prediction_token, dim=-1)
            next_token = torch.multinomial(probabilities, num_samples=1)
            next_token = next_token.item()

            autoregressive_seq.append(next_token)
        # get the autoregressive sequence
        return decode(autoregressive_seq)

In [30]:
model = GPT(n_layers)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

test_idx = torch.zeros(1, T).long()
model.forward(idx=test_idx)
decode(model.generate(idx=test_idx, max_new_tokens=100)[0].tolist())

torch.Size([64, 256, 8192])
tensor(9.3777, device='cuda:0', grad_fn=<NllLossBackward0>)


'!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!ownenta�ours cuentos las destro Jerry asustaralquier per something una estaban realized puer monkeys right trenMimiamaañanaice Com lloró Do pe patient rocas shoc élitaoking arms right weird tre van wouldn showingriba lazyater cansaron2cip ense mill shopkeeper llevó answ morder sido haría mug ignóm camas cosas penny sell nar Mi historiaEstas already dijoamesares Whatcíf t probó gard clum primos alrededor onto tocarlo songs jueM bet sí cocodr impatti night wild instrumentos ja daba cocodr pengu camera tuyo design palabra bowl pun'

In [31]:
# get the number of parameters in the model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("number of parameters in the model (millions): ", count_parameters(model) /1e6)

number of parameters in the model (millions):  12.817664


In [32]:
idx = torch.zeros(1, 1).long()
idx[:,-T:]

tensor([[0]], device='cuda:0')

In [33]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [34]:
eval_iters = 10
eval_interval = 300

@torch.no_grad()
def estimate_loss(is_last=False):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        real_iters = eval_iters
        if is_last and split == 'val':  # increase last eval to mitigate noise
            real_iters *= 10 
        losses = torch.zeros(real_iters)
        for k in range(real_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean() / chars_per_token
    model.train()
    return out

In [35]:
dump_model_interval = 1000
chars_per_token=3.9

#print('loading last model')
#model.load_state_dict(torch.load('tiny-stories-model-12.pt'))


for steps in tqdm.tqdm(range(0, max_iters)):
    xb, yb = get_batch('train')
    # loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)

    loss.backward()
    optimizer.step()
    if steps % eval_interval == 0:
        losses = estimate_loss()
        # wandb.log({"train": losses['train'].item(), "val": losses['val'].item(), "l2":l2})
        print({"train": losses['train'].item(), "val": losses['val'].item()})
    if steps % dump_model_interval == 0 and steps > 0:
        model_no = steps // dump_model_interval
        torch.save(model.state_dict(), f'tiny-stories-model-{model_no}.pt')

losses = estimate_loss(is_last=True)

  0%|          | 1/60000 [00:02<41:06:48,  2.47s/it]

{'train': 2.884622573852539, 'val': 2.884065866470337}


  1%|          | 301/60000 [01:58<16:58:50,  1.02s/it]

{'train': 0.6153015494346619, 'val': 0.6335249543190002}


  1%|          | 601/60000 [03:55<17:21:55,  1.05s/it]

...

 99%|█████████▉| 59401/60000 [6:32:55<10:27,  1.05s/it]

{'train': 0.25657400488853455, 'val': 0.255526602268219}


100%|█████████▉| 59701/60000 [6:34:53<05:10,  1.04s/it]

{'train': 0.2611274719238281, 'val': 0.25617286562919617}


100%|██████████| 60000/60000 [6:36:48<00:00,  2.52it/s]


In [ ]:
estimate_loss()

In [ ]:
torch.save(model.state_dict(), 'tiny-stories-model.pt')

In [ ]:
test_idx = torch.zeros(1, T).long() * 198
print(decode(
    model.generate(idx=test_idx, max_new_tokens=C)[0].tolist()
)[T:])